In [ ]:
%%shell
set -e

#---------------------------------------------------#
JULIA_VERSION="1.8.2" # any version ≥ 0.7.0
JULIA_PACKAGES="IJulia BenchmarkTools"
JULIA_PACKAGES_IF_GPU="CUDA" # or CuArrays for older Julia versions
JULIA_NUM_THREADS=2
#---------------------------------------------------#

if [ -z `which julia` ]; then
  # Install Julia
  JULIA_VER=`cut -d '.' -f -2 <<< "$JULIA_VERSION"`
  echo "Installing Julia $JULIA_VERSION on the current Colab Runtime..."
  BASE_URL="https://julialang-s3.julialang.org/bin/linux/x64"
  URL="$BASE_URL/$JULIA_VER/julia-$JULIA_VERSION-linux-x86_64.tar.gz"
  wget -nv $URL -O /tmp/julia.tar.gz # -nv means "not verbose"
  tar -x -f /tmp/julia.tar.gz -C /usr/local --strip-components 1
  rm /tmp/julia.tar.gz

  # Install Packages
  nvidia-smi -L &> /dev/null && export GPU=1 || export GPU=0
  if [ $GPU -eq 1 ]; then
    JULIA_PACKAGES="$JULIA_PACKAGES $JULIA_PACKAGES_IF_GPU"
  fi
  for PKG in `echo $JULIA_PACKAGES`; do
    echo "Installing Julia package $PKG..."
    julia -e 'using Pkg; pkg"add '$PKG'; precompile;"' &> /dev/null
  done

  # Install kernel and rename it to "julia"
  echo "Installing IJulia kernel..."
  julia -e 'using IJulia; IJulia.installkernel("julia", env=Dict(
      "JULIA_NUM_THREADS"=>"'"$JULIA_NUM_THREADS"'"))'
  KERNEL_DIR=`julia -e "using IJulia; print(IJulia.kerneldir())"`
  KERNEL_NAME=`ls -d "$KERNEL_DIR"/julia*`
  mv -f $KERNEL_NAME "$KERNEL_DIR"/julia  

  echo ''
  echo "Successfully installed `julia -v`!"
  echo "Please reload this page (press Ctrl+R, ⌘+R, or the F5 key) then"
  echo "jump to the 'Checking the Installation' section."
fi

In [ ]:
import Pkg
Pkg.add(["Graphs", "GraphPlot", "Colors"])

# Graphes

In [ ]:
using Graphs, GraphPlot, Colors

L'écosystème standard de graphes de Julia tourne autour de Graphs.jl : 

* Graphs.jl : graphe classique, sans poids ou métadonnées, dirigé ou non
* WeightedGraphs.jl : représentation efficace d'un graphe pondéré
* MetaGraphs.jl : représentation d'un graphe avec métadonnées au niveau des arêtes (comme des poids), des noeuds et du graphe
* MetaGraphsNext.jl : représentation plus efficace, mais pas encore finalisé

Sans oublier d'autres paquets pour des algorithmes plus spécifiques, comme des flots.

# Création d'un graphe

Un premier graphe assez simple, tout connecté : 

In [ ]:
g = complete_digraph(4)

In [ ]:
gplot(g, nodelabel=1:nv(g))

L'API de Graphs est assez simple à utiliser : `add_vertex!` pour ajouter un nœud, `add_edge!` pour une arête (dirigée ou non selon le type de graphe). Pour recréer le même graphe :

In [ ]:
g = DiGraph(0) # Graphe vide
for _ in 1:4
    add_vertex!(g)
end
for i in 1:4
    for j in 1:4
        if i != j
            add_edge!(g, i, j)
        end
    end
end

In [ ]:
gplot(g, nodelabel=1:nv(g))

# Analyse d'un graphe

On peut calculer différentes propriétés d'un graphe : `nv` pour obtenir le nombre de nœuds, `ne` pour le nombre d'arêtes.

In [ ]:
g = smallgraph("house")
gplot(g, nodelabel=1:nv(g), edgelabel=1:ne(g))

Par exemple, on peut chercher à déterminer à quel point un nœud est "central" dans le graphe :

In [ ]:
# betweenness_centrality : une mesure de centralité par nœud (vecteur)
# 1:nv(g) : itérateur sur la plage de nombres entre 1 et nv(g), comme range(1, nv) en Python
# zip(a, b) : itérateur sur les paires (a[i], b[i])
# collect(.) : vecteur à partir d'un itérateur, la liste n'est pas construite avant !
collect(zip(1:nv(g), betweenness_centrality(g)))

Utiliser cette info de centralité pour changer la taille des nœuds lors de l'affichage :

In [ ]:
gplot(g, nodelabel=1:nv(g), edgelabel=1:ne(g), nodesize=.15 .+ betweenness_centrality(g))

Ou leur couleur : 

In [ ]:
metrics = betweenness_centrality(g)
colours = [colorant"lightseagreen", colorant"orange"]
metric_class = [1 + Int(metrics[i] >= 0.15) for i in 1:nv(g)] # Vecteur : 1 si < 0.15, 2 si >= 0.15
colour_per_node = colours[metric_class] # Vecteur : valeur de colours, indice donné par metric_class
gplot(g, nodelabel=1:nv(g), edgelabel=1:ne(g), nodefillc=colour_per_node)